# 03a - CNN Fine-Tuning (Run 6a)
Fine-tune ResNet-18 layer4 as a per-frame classifier on Drive&Act Kinect IR.

**Pipeline:** Fine-tune CNN -> Re-extract features -> Train LSTM (notebook 03)

**Runtime:** GPU required. ~2-5 min/epoch, ~15 epochs max.

In [ ]:
# Colab Setup
import os
IN_COLAB = 'COLAB_GPU' in os.environ or os.path.exists('/content')

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    REPO_DIR = '/content/Driver-Activity-Recognition'
    if not os.path.exists(REPO_DIR):
        !git clone https://github.com/batuhne/Driver-Activity-Recognition.git {REPO_DIR}

    os.chdir(REPO_DIR)
    !pip install -q -r requirements.txt
    DATA_ROOT = '/content/drive/MyDrive/DriveAndAct'
else:
    DATA_ROOT = './data'

print(f'Working directory: {os.getcwd()}')
print(f'Data root: {DATA_ROOT}')

In [ ]:
import torch
from src.utils import load_config
from src.cnn_finetune import finetune_cnn

config = load_config()
if IN_COLAB:
    config['data']['root'] = DATA_ROOT
    drive_output = os.path.join(DATA_ROOT, 'results')
    config['output']['checkpoint_dir'] = os.path.join(drive_output, 'checkpoints')
    config['output']['log_dir'] = os.path.join(drive_output, 'logs')
    config['output']['figure_dir'] = os.path.join(drive_output, 'figures')

# ==================== Run 6a: CNN Fine-Tuning Config ====================
config['model']['freeze_mode'] = 'layer4'       # Only layer4 trainable

config['cnn_finetune'] = {
    'batch_size': 64,              # Single frames, can use larger batch
    'epochs': 15,
    'early_stop_patience': 7,
    'cnn_lr': 1e-4,               # CNN layer4 LR
    'fc_lr': 1e-3,                # Classifier head LR
    'weight_decay': 1e-4,
    'label_smoothing': 0.1,
    'gradient_clip': 1.0,
    'use_amp': True,
    'use_weighted_sampler': True,  # EN sampler for class balance
}
config['training']['en_beta'] = 0.99
config['training']['num_workers'] = 2

print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
print(f"\n--- CNN Fine-Tuning Config ---")
print(f"freeze_mode: {config['model']['freeze_mode']}")
print(f"batch_size: {config['cnn_finetune']['batch_size']}")
print(f"CNN LR: {config['cnn_finetune']['cnn_lr']}, FC LR: {config['cnn_finetune']['fc_lr']}")
print(f"epochs: {config['cnn_finetune']['epochs']}, patience: {config['cnn_finetune']['early_stop_patience']}")
print(f"AMP: {config['cnn_finetune']['use_amp']}")

In [ ]:
# Diagnostics: verify model structure
from src.cnn_finetune import CNNClassifier

model_check = CNNClassifier(num_classes=34, freeze_mode=config['model']['freeze_mode'])
total = sum(p.numel() for p in model_check.parameters())
trainable = sum(p.numel() for p in model_check.parameters() if p.requires_grad)
print(f'Total params: {total:,}')
print(f'Trainable params: {trainable:,}')
print(f'Frozen params: {total - trainable:,}')
print()
for idx, (name, child) in enumerate(model_check.backbone.features.named_children()):
    cp = sum(p.numel() for p in child.parameters())
    ct = sum(p.numel() for p in child.parameters() if p.requires_grad)
    if cp > 0:
        status = 'TRAINABLE' if ct > 0 else 'frozen'
        print(f'  [{idx}] {name:>2}: {cp:>10,} params  [{status}]')
del model_check

In [ ]:
# Run CNN fine-tuning
best_checkpoint = finetune_cnn(config)
print(f'\nBest checkpoint saved to: {best_checkpoint}')

In [ ]:
# GPU memory summary
if torch.cuda.is_available():
    peak_mem = torch.cuda.max_memory_allocated() / 1024**3
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'GPU peak memory: {peak_mem:.2f} GB / {total_mem:.1f} GB ({peak_mem/total_mem*100:.0f}%)')

## Re-extract Features with Fine-Tuned CNN
Now re-run feature extraction using the fine-tuned backbone.

In [ ]:
from src.feature_extract import extract_features

# Use fine-tuned features directory (don't overwrite original)
config['features']['save_dir'] = 'features_finetuned'
config['features']['dtype'] = 'float32'

# Path to fine-tuned CNN checkpoint
cnn_checkpoint = os.path.join(config['output']['checkpoint_dir'], 'cnn_finetuned.pth')
print(f'CNN checkpoint: {cnn_checkpoint}')
print(f'Output dir: {os.path.join(config["data"]["root"], config["features"]["save_dir"])}')

extract_features(config, cnn_checkpoint=cnn_checkpoint)

In [ ]:
# Verify re-extracted features
import numpy as np
import pandas as pd

feature_dir = os.path.join(config['data']['root'], config['features']['save_dir'])
for split in ['train', 'val', 'test']:
    manifest = pd.read_csv(os.path.join(feature_dir, split, 'manifest.csv'))
    sample_path = os.path.join(feature_dir, split, manifest.iloc[0]['filename'])
    sample = np.load(sample_path)
    print(f'{split}: {len(manifest)} segments, shape={sample.shape}, dtype={sample.dtype}')

print(f'\nFeatures saved to: {feature_dir}')
print('Now run notebook 03 (LSTM training) with features_dir = features_finetuned')